### Roughly patterned after LeNet-5
max instead of average pooling

In [1]:
function Base.conv2(A, B, shape::String)   # extend to accept "full" or "valid" as an third argument
    if shape == "full"
        return Base.conv2(A,B)
    elseif shape == "valid"
        ranges = [ min(a,b):max(a,b) for (a,b) in zip(size(A),size(B)) ]
        return Base.conv2(A,B)[ranges...]
    else
        error("shape must be full or valid")
    end
end

In [6]:
using Plots
gr()

# function for displaying a stack of images
# imgstack is mxnxp array that contains p images, each of which is mxn 
function montage(imgstack)
    plot(
        [heatmap(imgstack[:,:,i]) for i=1:size(imgstack,3)]...,
        legend=:none, axis=nothing     # options necessary to get nice spacing of the images
    )
end|

LoadError: syntax: incomplete: premature end of input

In [3]:
function maxpool(images)
    # 2x2 max pooling for set of images
    # returns MAXIMA, WINNERS
    # MAXIMA 2x smaller image, maximum of each 2x2 patch
    # WINNERS 2x smaller image, argmax (1..4) of each 2x2 patch

    m, n, k = size(images)  # mxn images, k of them
    # reshape to 2 x m/2 x 2 x n/2 x k
    maxima, winners = findmax(reshape(images, 2, div(m,2), 2, div(n,2), k), (1,3))
    return squeeze(maxima,(1,3)), squeeze(winners,(1,3))
end

function maxpoolback(delta_out, winners)
    # backprop through 2x2 max pooling
    #
    # DELTA_IN image, deltas
    # DELTA_OUT 2x smaller image, deltas
    # WINNERS 2x smaller image, argmax (1..4) of each 2x2 patch

    m, n, k = size(delta_out);  # mxn images, k of them
    delta_in = zeros(2*m,2*n,k)

    delta_in[winners] = delta_out
    return delta_in
end

maxpoolback (generic function with 1 method)

In [4]:
using MNIST

train, trainlabels = traindata()
train = reshape(train, 28, 28, size(train,2))/255.0
trainlabels = convert(Array{Int64,1},trainlabels)
trainlabels[trainlabels .== 0] = 10;  # tenth output signals a zero
niter = size(train, 3);  # number of training examples
nepoch = 10;  # number of epochs through training set

#f(x) = max(x,0)
#df(y) = float(y.>0)

f(x) = tanh(x)
df(y) = 1. - y.*y

epsinit = 0.1;   # scale of weight initialization
eta = 0.01;  # learning rate parameter

# initialize two convolution layers 
n1 = 6; n2 = 16;   # numbers of feature maps
w1 = epsinit*randn(5,5,n1);     # n1 kernels
w2 = epsinit*randn(5,5,n2,n1);  # n2 x n1 kernels

x0 = zeros(32, 32);     # input image
x1 = zeros(28, 28, n1);   # valid convolution by w1 reduces image size by 4
x1p = zeros(14, 14, n1);  # pooling reduces image size by 2x
x2 = zeros(10, 10, n2);   # convolution by w2 reduces image size by 4
x2p = zeros(5, 5, n2);    # pooling reduces image size by 2x

# initialize three fully connected layers
n3 = 120; n4 = 84; n5 = 10              # number of neurons per layer
W3 = epsinit*randn(n3,length(x2p[:]))   # 2D organization of x2p is discarded
W4 = epsinit*randn(n4,n3)
W5 = epsinit*randn(n5,n4);

10×84 Array{Float64,2}:
 -0.107835    0.119165   -0.0424434   …  -0.142116   -0.0061786  -0.0702819
 -0.0659408  -0.0787761   0.00185246      0.056534   -0.0221143   0.238002 
  0.0669089   0.0287926   0.135284        0.0972418   0.252342   -0.04662  
  0.0104491  -0.184309   -0.0895099       0.0868073   0.0815208  -0.173352 
  0.0877485  -0.0364961  -0.0897693      -0.153143   -0.108352    0.0986245
  0.170154    0.177363   -0.235362    …  -0.190661    0.0871033  -0.0196098
 -0.0346473  -0.0736787  -0.0369671      -0.21548     0.050761    0.239259 
  0.0874269   0.0617412   0.00655789     -0.124906   -0.108902    0.0984346
  0.267911    0.0828776   0.0333724      -0.0264601   0.0259854  -0.0679191
 -0.171294    0.1056     -0.0182364       0.0388453   0.0747895   0.0190685

In [ ]:
tic()
for iepoch = 1:nepoch
    errsq = zeros(niter)  # to monitor learning curve during epoch
    errcl = zeros(niter)
    for iter = 1:niter
        # zero pad 28x28 image to make it 32x32
        x0 = zeros(32,32);
        x0[3:30,3:30] = train[:,:,iter]

        for i = 1:n1
            x1[:,:,i] = conv2(x0, w1[:,:,i], "valid"); 
        end
        x1 = f(x1)
        x1p, x1w = maxpool(x1)

        x2 = zeros(size(x2));  # initialize to zero for accumulation
        for i = 1:n2
            for j = 1:n1
                x2[:,:,i] += conv2(x1p[:,:,j], w2[:,:,i,j], "valid");
            end
        end
        x2 = f(x2); 
        x2p, x2w = maxpool(x2)

        # discard 2D organization of x2p by reshaping to x2p(:)
        x3 = f(W3*x2p[:])
        x4 = f(W4*x3)
        x5 = f(W5*x4)
        prediction = indmax(x5);
        errcl[iter] = float(prediction != trainlabels[iter]);
        
        # backward pass
        d = -ones(n5,1); d[trainlabels[iter]] = 1;  # output vector
        err = d - x5; 
        delta5 = err.*df(x5);
        delta4 = (W5'*delta5).*df(x4)
        delta3 = (W4'*delta4).*df(x3)
        delta2p = W3'*delta3
        delta2p = reshape(delta2p, size(x2p))  # restore 2D organization
        delta2 = maxpoolback(delta2p, x2w).*df(x2)
        delta1p = zeros(size(x1p))
        for j = 1:n1
            for i = 1:n2
                delta1p[:,:,j] += conv2(delta2[:,:,i], w2[end:-1:1,end:-1:1,i,j], "full")
            end
        end
        delta1 = maxpoolback(delta1p, x1w).*df(x1)

        # weight updates
        W5 += eta*delta5*x4'
        W4 += eta*delta4*x3'
        W3 += eta*delta3*x2p[:]'
        for i = 1:n2
            for j = 1:n1
                w2[:,:,i,j] += eta*conv2(x1p[end:-1:1,end:-1:1,j], delta2[:,:,i], "valid")
            end
        end
        for i = 1:n1
            w1[:,:,i] += eta*conv2(x0[end:-1:1,end:-1:1], delta1[:,:,i], "valid")
        end
        if rem(iter,100) == 0
            toc()
            tic()
            IJulia.clear_output(true)
            plot(
                plot(cumsum(errsq[1:iter])./(1:iter)),
                plot(cumsum(errcl[1:iter])./(1:iter)),
                bar(x5),
                histogram(x4),
                histogram(x3),
                heatmap(x0,yflip=true),
                plot(
                    [heatmap(w1[:,:,i],
                             axis = nothing, 
                             color = :grays, 
                             yflip = true
                             ) for i = 1:n1]...), 
                layout = @layout [a b c; d e f; g{0.5h}]
            ) |> display
            sleep(0.01)
        end
    end
end

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 1200 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,92.4759 30.7367,92.4759 30.8698,92.4759 31.0028,92.4759 31.1359,92.4759 31.269,92.4759 31.402,92.4759 31.5351,92.4759 31.6681,92.4759 31.8012,92.4759 
 31.9342,92.4759 32.0673,92.4759 32.2003,92.4759 32.3334,92.4759 32.4664,92.4759 32.5995,92.4759 32.7326,92.4759 32.8656,92.4759 32.9987,92.4759 33.1317,92.4759 
 33.2648,92.4759 33.3978,92.4759 33.5309,92.4759 33.6639,92.4759 33.797,92.4759 33.9301,92.4759 34.0631,92.4759 34.1962,92.4759 34.3292,92.4759 34.4623,92.4759 
 34.5953,92.4759 34.7284,92.4759 34.8614,92.4759 34.9945,92.4759 35.1276,92.4759 35.2606,92.4759 35.3937,92.4759 35.5267,92.4759 35.6598,92.4759 35.7928,92.4759 
 35.9259,92.4759 36.0589,92.4759 36.192,92.4759 36.3251,92.4759 36.4581,92.4759 36.5912,92.4759 36.7242,92.4759 36.8573,92.4759 36.9903,92.4759 37.1234,92.4759 
 37.2564,92.4759 37.3895,92.4759 37.5226,92.4759 37.6556,92.4759 37.7887,92.4759 37.9217,92.4759 38.0548,92.4759 38.1878,92.4759 38.3209,92.4759 38.4539,92.4759 
 38.587,92.4759 38.7201,92.4759 38.8531,92.4759 38.9862,92.4759 39.1192,92.4759 39.2523,92.4759 39.3853,92.4759 39.5184,92.4759 39.6514,92.4759 39.7845,92.4759 
 39.9176,92.4759 40.0506,92.4759 40.1837,92.4759 40.3167,92.4759 40.4498,92.4759 40.5828,92.4759 40.7159,92.4759 40.8489,92.4759 40.982,92.4759 41.115,92.4759 
 41.2481,92.4759 41.3812,92.4759 41.5142,92.4759 41.6473,92.4759 41.7803,92.4759 41.9134,92.4759 42.0464,92.4759 42.1795,92.4759 42.3125,92.4759 42.4456,92.4759 
 42.5787,92.4759 42.7117,92.4759 42.8448,92.4759 42.9778,92.4759 43.1109,92.4759 43.2439,92.4759 43.377,92.4759 43.51,92.4759 43.6431,92.4759 43.7762,92.4759 
 43.9092,92.4759 44.0423,92.4759 44.1753,92.4759 44.3084,92.4759 44.4414,92.4759 44.5745,92.4759 44.7075,92.4759 44.8406,92.4759 44.9737,92.4759 45.1067,92.4759 
 45.2398,92.4759 45.3728,92.4759 45.5059,92.4759 45.6389,92.4759 45.772,92.4759 45.905,92.4759 46.0381,92.4759 46.1712,92.4759 46.3042,92.4759 46.4373,92.4759 
 46.5703,92.4759 46.7034,92.4759 46.8364,92.4759 46.9695,92.4759 47.1025,92.4759 47.2356,92.4759 47.3687,92.4759 47.5017,92.4759 47.6348,92.4759 47.7678,92.4759 
 47.9009,92.4759 48.0339,92.4759 48.167,92.4759 48.3,92.4759 48.4331,92.4759 48.5661,92.4759 48.6992,92.4759 48.8323,92.4759 48.9653,92.4759 49.0984,92.4759 
 49.2314,92.4759 49.3645,92.4759 49.4975,92.4759 49.6306,92.4759 49.7636,92.4759 49.8967,92.4759 50.0298,92.4759 50.1628,92.4759 50.2959,92.4759 50.4289,92.4759 
 50.562,92.4759 50.695,92.4759 50.8281,92.4759 50.9611,92.4759 51.0942,92.4759 51.2273,92.4759 51.3603,92.4759 51.4934,92.4759 51.6264,92.4759 51.7595,92.4759 
 51.8925,92.4759 52.0256,92.4759 52.1586,92.4759 52.2917,92.4759 52.4248,92.4759 52.5578,92.4759 52.6909,92.4759 52.8239,92.4759 52.957,92.4759 53.09,92.4759 
 53.2231,92.4759 53.3561,92.4759 53.4892,92.4759 53.6223,92.4759 53.7553,92.4759 53.8884,92.4759 54.0214,92.4759 54.1545,92.4759 54.2875,92.4759 54.4206,92.4759 
 54.5536,92.4759 54.6867,92.4759 54.8198,92.4759 54.9528,92.4759 55.0859,92.4759 55.2189,92.4759 55.352,92.4759 55.485,92.4759 55.6181,92.4759 55.7511,92.4759 
 55.8842,92.4759 56.0173,92.4759 56.1503,92.4759 56.2834,92.4759 56.4164,92.4759 56.5495,92.4759 56.6825,92.4759 56.8156,92.4759 56.9486,92.4759 57.0817,92.4759 
 57.2147,92.4759 57.3478,92.4759 57.4809,92.4759 57.6139,92.4759 57.747,92.4759 57.88,92.4759 58.0131,92.4759 58.1461,92.4759 58.2792,92.4759 58.4122,92.4759 
 58.5453,92.4759 58.6784,92.4759 58.8114,92.4759 58.9445,92.4759 59.0775,92.4759 59.2106,92.4759 59.3436,92.4759 59.4767,92.4759 59.6097,92.4759 59.7428,92.4759 
 59.8759,92.4759 60.0089,92.4759 60.142,92.4759 60.275,92.4759 60.4081,92.4759 60.5411,92.4759 60.6